# Semantic QA over tables

https://docs.llamaindex.ai/en/stable/examples/query_engine/sec_tables/tesla_10q_table.html#extract-elements

In [7]:
from pydantic import BaseModel
from unstructured.partition.html import partition_html
import pandas as pd

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

In [8]:
# load pdfs
from llama_index import SimpleDirectoryReader
from llama_index.readers.file.flat_reader import FlatReader

reader = SimpleDirectoryReader(
    input_files=["../data/apple/AAPL.pdf"]
)

aapl = reader.load_data()

reader = SimpleDirectoryReader(
    input_files=["../data/meta/meta.pdf"]
)
meta = reader.load_data()

# reader  = FlatReader()
# docs1 = reader.load_data(Path("../data/apple/AAPL.pdf"))

In [9]:

from platform import node
from llama_index.node_parser import UnstructuredElementNodeParser

node_parser = UnstructuredElementNodeParser()

aapl_nodes = node_parser.get_nodes_from_documents(aapl, show_progress=True)
meta_nodes = node_parser.get_nodes_from_documents(meta, show_progress=True)

0it [00:00, ?it/s]into nodes:   0%|          | 0/80 [00:00<?, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]into nodes:   8%|▊         | 6/80 [00:00<00:01, 57.57it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]into nodes:  15%|█▌        | 12/80 [00:00<00:01, 47.90it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]into nodes:  21%|██▏       | 17/80 [00:00<00:01, 46.16it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]into nodes:  29%|██▉       | 23/80 [00:00<00:01, 50.73it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]into nodes:  38%|███▊      | 30/80 [00:00<00:00, 56.65it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00

In [10]:
aapl_nodes[0]

TextNode(id_='757c8376-3de4-423b-9077-98be72f80e97', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='daaf81fd-b981-4e8f-a144-9df292f526ba', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='24ca0de2fd3ae1bdca38a1ee34282977f0f86e643dc38884a5dbe9cb0086cfbf')}, hash='24ca0de2fd3ae1bdca38a1ee34282977f0f86e643dc38884a5dbe9cb0086cfbf', text='UNITED STATES\nSECURITIES AND EXCHANGE COMMISSION\nWashington, D.C. 20549\nFORM 10-K\n(Mark One)\n☒    ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\nFor the fiscal year ended September\xa024, 2022\nor\n☐    TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\nFor the transition period from \xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0  to \xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 .\nCommission File Number: 001-36743\nApple Inc.\n(Exact name of Reg

In [11]:
aapl_base_nodes, aapl_node_mappings = node_parser.get_base_nodes_and_mappings(aapl_nodes)
meta_base_nodes, meta_node_mappings = node_parser.get_base_nodes_and_mappings(meta_nodes)

In [19]:
aapl_base_nodes[30].text

'Company Stock Performance\nThe following graph  shows a comparison of cumulative total shareholder return, calculated on a dividend-reinvested basis, for \nthe Company, the S&P 500 Index, the S&P Information Technology Index and the Dow Jones U.S. Technology Supersector \nIndex for the five years ended September\xa024, 2022 . The graph assumes $100 was invested in each of the Company’s common \nstock, the S&P 500 Index, the S&P Information Technology Index and the Dow Jones U.S. Technology Supersector Index as of \nthe market close on September\xa0 29, 2017 . Past stock price performance is not necessarily indicative of future stock price \nperformance .\nCOMPARISON OF 5-YEAR CUMULATIVE TOTAL RETURN*Among Apple Inc., the S&P 500 Index, the S&P Information Technology Index and the Dow Jones U.S. Technology Supersector IndexApple Inc.S&P 500 IndexS&P Information Technology IndexDow Jones U.S. Technology Supersector Index9/29/179/29/189/28/199/26/209/25/219/24/22$0$100$200$300$400$500\n*

In [20]:
from llama_index.retrievers import RecursiveRetriever
from llama_index.query_engine import RetrieverQueryEngine
from llama_index import VectorStoreIndex

vector_index = VectorStoreIndex(aapl_base_nodes)
vector_retriever = vector_index.as_retriever(similarity_top_k=1)
vector_query_engine = vector_index.as_query_engine(similarity_top_k=1)

from llama_index.retrievers import RecursiveRetriever

recursive_retriever = RecursiveRetriever(
    "vector",
    retriever_dict={"vector": vector_retriever},
    node_dict=aapl_node_mappings,
    verbose=True,
)
query_engine = RetrieverQueryEngine.from_args(recursive_retriever)

In [25]:
response = query_engine.query("what was the management's discussion and analysis")
print(str(response))

Retrieving with query id None: what was the management's discussion and analysis
Retrieving text node: Item 7. Management’s Discussion and Analysis of Financial Condition and Results of Operations
The following discussion should be read in conjunction with the consolidated financial statements and accompanying notes 
included in Part II, Item 8 of this Form 10-K. This section of this Form 10-K generally discusses 2022  and 2021  items and year-
to-year comparisons between 2022  and 2021 . Discussions of 2020  items and year-to-year comparisons between 2021  and 
2020  are not included in this Form 10-K, and can be found in “Management’s Discussion and Analysis of Financial Condition 
and Results of Operations” in Part II, Item 7 of the Company’s Annual Report on Form 10-K for the fiscal year ended 
September 25, 2021 .
Fiscal Year Highlights
Fiscal 2022  Highlights
Total net sales increased  8% or $28.5 billion  during 2022  compared to 2021 , driven primarily by higher net sales of iP

In [34]:
import os
from dotenv import load_dotenv

env_file_path = "../.env"
load_dotenv(dotenv_path=env_file_path)
api_key = os.environ.get("OPENAI_API_KEY")

In [35]:
from llama_index.tools import QueryEngineTool, ToolMetadata
from llama_index.query_engine import SubQuestionQueryEngine

from llama_index import ServiceContext
from llama_index.llms import OpenAI

import nest_asyncio

nest_asyncio.apply()

llm = OpenAI(model="gpt-3.5-turbo", api_key=api_key)
service_context = ServiceContext.from_defaults(llm=llm)

In [36]:
query_engine_tool = [
    QueryEngineTool(
        query_engine=query_engine,
        metadata=ToolMetadata(
            name = "Apple",
            description="provides information about Apple financials for the year 2022",
        ),
    )
]

sub_query_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=query_engine_tool,
    service_context=service_context,
    use_async=True
)

In [40]:
response = sub_query_engine.query("Provide highlights from the management's discussion and analysis")
print("-"*50)
print(str(response))

Generated 1 sub questions.
[Apple] Q: What are the key points mentioned in the management's discussion and analysis?
Retrieving with query id None: What are the key points mentioned in the management's discussion and analysis?
Retrieving text node: Item 7. Management’s Discussion and Analysis of Financial Condition and Results of Operations
The following discussion should be read in conjunction with the consolidated financial statements and accompanying notes 
included in Part II, Item 8 of this Form 10-K. This section of this Form 10-K generally discusses 2022  and 2021  items and year-
to-year comparisons between 2022  and 2021 . Discussions of 2020  items and year-to-year comparisons between 2021  and 
2020  are not included in this Form 10-K, and can be found in “Management’s Discussion and Analysis of Financial Condition 
and Results of Operations” in Part II, Item 7 of the Company’s Annual Report on Form 10-K for the fiscal year ended 
September 25, 2021 .
Fiscal Year Highlights


In [41]:
print(str(response))

The management's discussion and analysis highlights include an increase in total net sales in 2022 compared to 2021, driven by higher net sales of iPhone, Services, and Mac. The weakness in foreign currencies relative to the U.S. dollar had a negative impact on all Products and Services net sales. The company also made significant product, service, and software announcements throughout fiscal 2022, including updates to MacBook Pro, AirPods, iPhone, Mac Studio, Studio Display, iPad Air, MacBook Air, MacBook Pro, operating systems, and the introduction of Apple Pay Later. Additionally, the company announced an increase in its Program authorization and raised its quarterly dividend. The COVID-19 pandemic has had a significant impact on the company's business, results of operations, and financial condition, including disruptions in supply chains.
